<a href="https://colab.research.google.com/github/Varun-Mulchandani/Reddit-Flair-Classifier/blob/master/Initial_baseline_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install bert-for-tf2
!pip install sentencepiece

     |████████████████████████████████| 40kB 1.0MB/s 
  Created wheel for bert-for-tf2: filename=bert_for_tf2-0.14.1-cp36-none-any.whl size=30083 sha256=e1a588108c799fa44c480c9785384221c9f13bca04d05859d3a22a3ced938c85
  Stored in directory: /root/.cache/pip/wheels/dd/f1/10/861fd7899727e4034293fb1dfef45b00f8cd476d21d3b3821e
  Created wheel for py-params: filename=py_params-0.9.7-cp36-none-any.whl size=7302 sha256=764c01cb90c9702462e52fd04ac04dc57531fc770ecfafa5f706a75713b1105b
  Stored in directory: /root/.cache/pip/wheels/67/f5/19/b461849a50aefdf4bab47c4756596e82ee2118b8278e5a1980
  Created wheel for params-flow: filename=params_flow-0.8.0-cp36-none-any.whl size=15999 sha256=09f857d97feb345272762ca0e648c28f23692bf1f251f652365946f87672a8d5
  Stored in directory: /root/.cache/pip/wheels/88/41/05/1a9955d1d01575bbd58aab76e22f8c7eeabba905d551576f43
Successfully built bert-for-tf2 py-params params-flow
     |████████████████████████████████| 1.0MB 1.4MB/s 


In [0]:
import math

In [0]:
import pandas as pd
import numpy as np
import regex as re

In [0]:
try:
  %tensorflow_version 2.x
except Exception:
  pass
import tensorflow as tf
import tensorflow_hub as hub
from tensorflow.keras import layers
import bert

In [0]:
data = pd.read_csv('reddddddit.csv')

In [6]:
data.head()

,author,authors,body,comment,comms_num,created,flair,id,score,title,url,combined_features
0,dhavalcoholic,ICICIPruLifeIns,reposting lack activity r askindiahello last y...,dear policy holder dhavalcoholic request help ...,1,1.386254e+09,AskIndia,1s57oi,1,need feedback insurance policy took xpost aski...,https://www.reddit.com/r/india/comments/1s57oi...,need feedback insurance policy took xpost aski...
1,amitkumarthakur,RAD-Business RAD-Business None barcam10 _snor...,24hrs local police station register case dont ...,calm downgo sp office town file grievance imme...,24,1.554080e+09,AskIndia,b7pvwt,94,somebody want kill full family,https://www.reddit.com/r/india/comments/b7pvwt...,somebody want kill full familycalm downgo sp o...
2,FrustratedOCIHopeful,plshelpthedog ayyylmaaaoo Proper_Boysenberry ...,hello askindia first time poster long time lur...,honestly supervisor behaved exactly government...,27,1.555361e+09,AskIndia,bdfid1,10,ambassador india takes back newly issued oci c...,https://www.reddit.com/r/india/comments/bdfid1...,ambassador india takes back newly issued oci c...
3,aloo_vs_bhaloo,vcdarklord tilismilis aloo_vs_bhaloo dogaa fo...,r tooafraidtoask india edition,modi control sex desires jerk someone else pro...,22,1.566529e+09,AskIndia,cu1xn4,18,randians afraid ask,https://www.reddit.com/r/india/comments/cu1xn4...,randians afraid askmodi control sex desires je...
4,multubunu,NaN,hello submitted r raskindia week ago got answe...,NaN,0,1.361085e+09,AskIndia,18ntue,0,askindia cingari cengar tzengar,https://www.reddit.com/r/india/comments/18ntue...,askindia cingari cengar tzengarhttps://www.red...


In [7]:
data.shape

(1518, 12)

In [0]:
def preprocess_text(sentence):
  sentence = remove_tags(sentence)
  sentence = re.sub('[^a-zA-Z]', ' ', sentence)
  sentence = re.sub(r'\s+[a-zA-Z]\s+', ' ', sentence)
  sentence = re.sub(r'\s+', ' ', sentence)
  return sentence
TAG_RE = re.compile(r'<[^>]+>')
def remove_tags(text):
  return TAG_RE.sub('', text)

In [9]:
print(type(list(data['title'])[0]))

<class 'str'>


In [0]:
cleaned_titles = []
sentences = list(data['title'])
for sentence in sentences:
  cleaned_titles.append(preprocess_text(sentence))

In [0]:
#cleaned_titles

In [0]:
#for i in list(data['body']):
#  print(type(i))
#body can be both str or float whereas in title we have only str

In [0]:
cleaned_bodies = []
sentences = list(data['body'])
for sentence in sentences:
  if type(sentence) == float:#as some of the bodies are empty
    cleaned_bodies.append(str(sentence))
  else:
    cleaned_bodies.append(preprocess_text(sentence))

In [0]:
#cleaned_bodies

Creating the BERT tokenizer

In [0]:
BertTokenizer = bert.bert_tokenization.FullTokenizer
bert_layer = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/1", trainable = False)
vocabulary_file = bert_layer.resolved_object.vocab_file.asset_path.numpy()
to_lower_case = bert_layer.resolved_object.do_lower_case.numpy()
tokenizer = BertTokenizer(vocabulary_file, to_lower_case)

In [16]:
tokenizer.convert_tokens_to_ids(tokenizer.tokenize("sep"))

[19802]

In [0]:
#establishing tokens for a sentence to start or begin and a seperator for the title and the body
start_token = len(tokenizer.vocab) + 1
end_token = len(tokenizer.vocab) + 2
seperator_token = tokenizer.convert_tokens_to_ids(tokenizer.tokenize("sep"))[0]
new_vocab_size = len(tokenizer.vocab) + 2

In [0]:
final_input = []
for (title, body) in zip(cleaned_titles, cleaned_bodies):
  input_i = tokenizer.convert_tokens_to_ids(tokenizer.tokenize(title)) + [seperator_token] + tokenizer.convert_tokens_to_ids(tokenizer.tokenize(body))
  final_input.append(input_i)
#final_input[0]

In [19]:
flairs = []
for i in list(data['flair']):
  if i in flairs:
    pass
  else:
    flairs.append(i)
print(flairs)
count = 0
label_to_id = {}
for i in flairs:
  label_to_id[i] = count
  count += 1
print(label_to_id)

['AskIndia', 'Non-Political', '[R]eddiquette', 'Scheduled', 'Photography', 'Science/Technology', 'Politics', 'Business/Finance', 'Policy/Economy', 'Sports', 'Food', 'AMA']
{'AskIndia': 0, 'Non-Political': 1, '[R]eddiquette': 2, 'Scheduled': 3, 'Photography': 4, 'Science/Technology': 5, 'Politics': 6, 'Business/Finance': 7, 'Policy/Economy': 8, 'Sports': 9, 'Food': 10, 'AMA': 11}


In [0]:
final_input_len = [[inputs, label_to_id[list(data['flair'])[i]], len(inputs)] for i, inputs in enumerate(final_input)]
#final_input_len

In [0]:
#for i in list(data['flair']):
#  print(label_to_id[i])

In [0]:
import random
random.shuffle(final_input_len)

In [0]:
final_input_len.sort(key=lambda x: x[2])

In [0]:
sorted_input_labels = [(inputs_lab[0], inputs_lab[1]) for inputs_lab in final_input_len]

In [0]:
processed_dataset = tf.data.Dataset.from_generator(lambda: sorted_input_labels, output_types = (tf.int32, tf.int32))

In [0]:
BATCH_SIZE = 32
batched_dataset = processed_dataset.padded_batch(BATCH_SIZE, padded_shapes = ((None,), ()))

In [27]:
next(iter(batched_dataset))

(<tf.Tensor: shape=(32, 6), dtype=int32, numpy=
 array([[14288, 19802, 16660,     0,     0,     0],
        [14288, 19802, 16660,     0,     0,     0],
        [ 2197,  3345, 19802, 16660,     0,     0],
        [26102,  2634, 19802, 16660,     0,     0],
        [ 5675,  4599, 19802, 16660,     0,     0],
        [ 2197,  3345, 19802, 16660,     0,     0],
        [ 3835,  5466, 19802, 16660,     0,     0],
        [ 3403,  2767, 19802, 16660,     0,     0],
        [ 3403,  2767, 19802, 16660,     0,     0],
        [23556,  4839, 19802, 16660,     0,     0],
        [ 4608,  2397, 19802, 16660,     0,     0],
        [ 5855,  8660,  6569, 19802, 16660,     0],
        [ 6975,  4368,  2455, 19802, 16660,     0],
        [ 3047,  7483,  1054, 19802, 16660,     0],
        [10958, 21886, 12897, 19802, 16660,     0],
        [18465,  5855,  8955, 19802, 16660,     0],
        [ 5855,  8660,  6569, 19802, 16660,     0],
        [ 2296,  2309,  2051, 19802, 16660,     0],
        [18465, 

In [0]:
TOTAL_BATCHES = math.ceil(len(sorted_input_labels)/BATCH_SIZE)
TEST_BATCHES = TOTAL_BATCHES // 10
batched_dataset.shuffle(TOTAL_BATCHES)
test_data = batched_dataset.take(TEST_BATCHES)
train_data = batched_dataset.skip(TEST_BATCHES)

In [0]:
class TEXT_MODEL(tf.keras.Model):
  def __init__(self,
               vocabulary_size,
               embedding_dimensions = 128,
               cnn_filters = 50,
               dnn_units = 512,
               model_output_classes = 12,
               dropout_rate = 0.1,
               training = False,
               name = 'text_model'):
    super(TEXT_MODEL, self).__init__(name=name)
        
    self.embedding = layers.Embedding(vocabulary_size,
                                          embedding_dimensions)
    self.cnn_layer1 = layers.Conv1D(filters=cnn_filters,
                                        kernel_size=2,
                                        padding="valid",
                                        activation="relu")
    self.cnn_layer2 = layers.Conv1D(filters=cnn_filters,
                                        kernel_size=3,
                                        padding="valid",
                                        activation="relu")
    self.cnn_layer3 = layers.Conv1D(filters=cnn_filters,
                                        kernel_size=4,
                                        padding="valid",
                                        activation="relu")
    self.pool = layers.GlobalMaxPool1D()
        
    self.dense_1 = layers.Dense(units=dnn_units, activation="relu")
    self.dropout = layers.Dropout(rate=dropout_rate)
    self.last_dense = layers.Dense(units=model_output_classes,
                                           activation="softmax")
    
  def call(self, inputs, training):
    l = self.embedding(inputs)
    l_1 = self.cnn_layer1(l) 
    l_1 = self.pool(l_1) 
    l_2 = self.cnn_layer2(l) 
    l_2 = self.pool(l_2)
    l_3 = self.cnn_layer3(l)
    l_3 = self.pool(l_3) 
        
    concatenated = tf.concat([l_1, l_2, l_3], axis=-1) # (batch_size, 3 * cnn_filters)
    concatenated = self.dense_1(concatenated)
    concatenated = self.dropout(concatenated, training)
    model_output = self.last_dense(concatenated)
        
    return model_output

In [0]:
VOCAB_LENGTH = len(tokenizer.vocab)
EMB_DIM = 200
CNN_FILTERS = 100
DNN_UNITS = 256
OUTPUT_CLASSES = 12

DROPOUT_RATE = 0.5

NB_EPOCHS = 5

In [0]:
text_model = TEXT_MODEL(vocabulary_size=VOCAB_LENGTH,
                        embedding_dimensions=EMB_DIM,
                        cnn_filters=CNN_FILTERS,
                        dnn_units=DNN_UNITS,
                        model_output_classes=OUTPUT_CLASSES,
                        dropout_rate=DROPOUT_RATE)

In [0]:
text_model.compile(loss = 'sparse_categorical_crossentropy', optimizer = 'adam', metrics = ['sparse_categorical_accuracy'])

In [33]:
text_model.fit(train_data,epochs = NB_EPOCHS)

Epoch 1/5
44/44 [==============================] - 8s 187ms/step - loss: 2.3122 - sparse_categorical_accuracy: 0.2252
Epoch 2/5
44/44 [==============================] - 8s 182ms/step - loss: 1.8359 - sparse_categorical_accuracy: 0.4050
Epoch 3/5
44/44 [==============================] - 8s 183ms/step - loss: 1.2745 - sparse_categorical_accuracy: 0.6410
Epoch 4/5
44/44 [==============================] - 8s 182ms/step - loss: 0.7872 - sparse_categorical_accuracy: 0.8252
Epoch 5/5
44/44 [==============================] - 8s 184ms/step - loss: 0.4032 - sparse_categorical_accuracy: 0.9245


In [34]:
results = text_model.evaluate(test_data)
print(results)

4/4 [==============================] - 0s 8ms/step - loss: 1.5624 - sparse_categorical_accuracy: 0.5156
[1.5624210834503174, 0.515625]
